In [13]:
import aocd
import re
import numpy as np
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2021, day=16)
data = list(puzzle.input_data)

In [14]:
packet = "".join([bin(int(x, 16))[2:].zfill(4) for x in data])

In [15]:
def parse_packets(packet, versions, values, num_packets, num_bits):
    packets_parsed = 0
    bits_parsed = 0
    
    while(packets_parsed != num_packets and bits_parsed != num_bits):
        sub_values = []
        version = packet[0:3]
        versions.append(version)
        type_id = int(packet[3:6], 2)

        if(type_id == 4):
            total = ""
            parts = chunks(packet[6:], 5)
            for part in parts:
                total += part[1:5]
                if(part[0] == "0"):
                    break
            sub_values.append(int(total, 2))
            bits_used = (len(total)//4) * 5 + 6
            
        else:
            if(packet[6] == "0"):
                length = int(packet[7:22], 2)
                parse_packets(packet[22:22+length], versions, sub_values, -1, length)
                bits_used = 22+length
            else:
                length = int(packet[7:18], 2)
                retval = parse_packets(packet[18:], versions, sub_values, length, -1)
                bits_used = 18+retval

        bits_parsed += bits_used
        packets_parsed += 1
        packet = packet[bits_used:]
                
        if(type_id == 0):
            values.append(sum(sub_values))
        elif(type_id == 1):
            values.append(np.prod(sub_values))
        elif(type_id == 2):
            values.append(min(sub_values))
        elif(type_id == 3):
            values.append(max(sub_values))
        elif(type_id == 4):
            values.append(sub_values[0])
        elif(type_id == 5):
            values.append(1 if sub_values[0] > sub_values[1] else 0)
        elif(type_id == 6):
            values.append(1 if sub_values[0] < sub_values[1] else 0)
        elif(type_id == 7):
            values.append(1 if sub_values[0] == sub_values[1] else 0)
        
    return bits_parsed
                

In [16]:
versions = []
values = []
retval = parse_packets(packet, versions, values, 1, -1)

In [17]:
puzzle.answer_a = sum([int(x, 2) for x in versions])

In [18]:
puzzle.answer_b = str(values[0])